# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [1]:
# --- Imports and path setup ---
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

# Ensure local package is importable

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_with_fakes_like_input.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

CSV exists: True
Output dir: ../twilight_outputs


## Configure the planner

The next cell defines a complete `PlannerConfig`. Every parameter of the
dataclass is included so the notebook doubles as documentation. Edit any value
to explore different strategies or hardware assumptions.


In [2]:
# ---- User-editable parameters ----

# Date range (UTC)
START_DATE = "2023-01-01"
END_DATE = "2024-01-1"
RUN_LABEL = None

# -- Site --
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

# -- Visibility --
MIN_ALT_DEG = 20.0
TWILIGHT_SUN_ALT_MIN_DEG = -14.0  # minimum altitude for twilight
TWILIGHT_SUN_ALT_MAX_DEG = -8.0  # maximum altitude for twilight


# -- Filters and hardware --
FILTERS = ["g", "r", "i", "z"]
CAROUSEL_CAPACITY = 5
FILTER_CHANGE_S = 120.0
READOUT_S = 2.0
FILTER_CHANGE_TIME_S = None  # legacy alias
READOUT_TIME_S = None  # legacy alias
INTER_EXPOSURE_MIN_S = 15.0
EXPOSURE_BY_FILTER = {"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0}
# NEW — cadence knobs
CADENCE_ENABLE = True
CADENCE_PER_FILTER = True
CADENCE_DAYS_TARGET = 3.0
CADENCE_JITTER_DAYS = 0.25
CADENCE_DAYS_TOLERANCE = 0.5
CADENCE_BONUS_SIGMA_DAYS = 0.5
CADENCE_BONUS_WEIGHT = 0.25

# Allow colors in one visit (if you want them)
MAX_FILTERS_PER_VISIT = 3  # was 1
ALLOW_FILTER_CHANGES_IN_TWILIGHT = True  # was False
START_FILTER = FILTERS[0]
SUN_ALT_POLICY = [
    (-18.0, -15.0, ["g", "r", "i", "z", "y"]),
    (-15.0, -12.0, ["r", "i", "z", "y"]),
    (-12.0, 0.0, ["i", "r", "z", "y"]),
]

# -- Slew model --
SLEW_SMALL_DEG = 3.5
SLEW_SMALL_TIME_S = 4.0
SLEW_RATE_DEG_PER_S = 5.25
SLEW_SETTLE_S = 1.0

# -- Moon --
MIN_MOON_SEP_BY_FILTER = {
    "u": 80.0,
    "g": 50.0,
    "r": 35.0,
    "i": 30.0,
    "z": 25.0,
    "y": 20.0,
}
REQUIRE_SINGLE_TIME_FOR_ALL = True

# -- Time caps --
PER_SN_CAP_S = 120.0
MORNING_CAP_S = 'auto'
EVENING_CAP_S = 'auto'
TWILIGHT_STEP_MIN = 2
MAX_SN_PER_NIGHT = 99999

# -- Priority strategy --
PRIORITY_STRATEGY = "hybrid"  # or "lc"
HYBRID_DETECTIONS = 2
HYBRID_EXPOSURE = 300.0
LC_DETECTIONS = 90
LC_EXPOSURE = 99999.0

# -- Photometry / sky --
PIXEL_SCALE_ARCSEC = 0.2
ZPT1S = None
K_M = None
FWHM_EFF = None
READ_NOISE_E = 6.0
GAIN_E_PER_ADU = 1.0
ZPT_ERR_MAG = 0.01
DARK_SKY_MAG = None
TWILIGHT_DELTA_MAG = 2.5  # fallback if rubin_sim.skybrightness unavailable

# -- SIMLIB output --
SIMLIB_OUT = f"twilight_{START_DATE}_to_{END_DATE}.simlib"
SIMLIB_SURVEY = "LSST"
SIMLIB_FILTERS = "grizy"
SIMLIB_PIXSIZE = 0.2
SIMLIB_NPE_PIXEL_SATURATE = 8e5
SIMLIB_PHOTFLAG_SATURATE = 4096
SIMLIB_PSF_UNIT = "arcsec"

# -- Miscellaneous --
TYPICAL_DAYS_BY_TYPE = {
    "Ia": 60,
    "II-P": 90,
    "II-L": 60,
    "IIn": 110,
    "IIb": 60,
    "Ib": 50,
    "Ic": 50,
}
DEFAULT_TYPICAL_DAYS = 60
RA_COL = None
DEC_COL = None
DISC_COL = None
NAME_COL = None
TYPE_COL = None

# Assemble the configuration object with all parameters exposed
cfg = PlannerConfig(
    lat_deg=LAT_DEG,
    lon_deg=LON_DEG,
    height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    twilight_sun_alt_min_deg=TWILIGHT_SUN_ALT_MIN_DEG,
    twilight_sun_alt_max_deg=TWILIGHT_SUN_ALT_MAX_DEG,
    filters=FILTERS,
    carousel_capacity=CAROUSEL_CAPACITY,
    filter_change_s=FILTER_CHANGE_S,
    readout_s=READOUT_S,
    filter_change_time_s=FILTER_CHANGE_TIME_S,
    readout_time_s=READOUT_TIME_S,
    inter_exposure_min_s=INTER_EXPOSURE_MIN_S,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    max_filters_per_visit=MAX_FILTERS_PER_VISIT,
    start_filter=START_FILTER,
    sun_alt_policy=SUN_ALT_POLICY,
    slew_small_deg=SLEW_SMALL_DEG,
    slew_small_time_s=SLEW_SMALL_TIME_S,
    slew_rate_deg_per_s=SLEW_RATE_DEG_PER_S,
    slew_settle_s=SLEW_SETTLE_S,
    min_moon_sep_by_filter=MIN_MOON_SEP_BY_FILTER,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
    per_sn_cap_s=PER_SN_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    evening_cap_s=EVENING_CAP_S,
    twilight_step_min=TWILIGHT_STEP_MIN,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    cadence_enable=CADENCE_ENABLE,
    cadence_per_filter=CADENCE_PER_FILTER,
    cadence_days_target=CADENCE_DAYS_TARGET,
    cadence_jitter_days=CADENCE_JITTER_DAYS,
    cadence_days_tolerance=CADENCE_DAYS_TOLERANCE,
    cadence_bonus_sigma_days=CADENCE_BONUS_SIGMA_DAYS,
    cadence_bonus_weight=CADENCE_BONUS_WEIGHT,
    hybrid_detections=HYBRID_DETECTIONS,
    hybrid_exposure_s=HYBRID_EXPOSURE,
    lc_detections=LC_DETECTIONS,
    lc_exposure_s=LC_EXPOSURE,
    priority_strategy=PRIORITY_STRATEGY,
    pixel_scale_arcsec=PIXEL_SCALE_ARCSEC,
    zpt1s=ZPT1S,
    k_m=K_M,
    fwhm_eff=FWHM_EFF,
    read_noise_e=READ_NOISE_E,
    gain_e_per_adu=GAIN_E_PER_ADU,
    zpt_err_mag=ZPT_ERR_MAG,
    dark_sky_mag=DARK_SKY_MAG,
    twilight_delta_mag=TWILIGHT_DELTA_MAG,
    simlib_out=SIMLIB_OUT,
    simlib_survey=SIMLIB_SURVEY,
    simlib_filters=SIMLIB_FILTERS,
    simlib_pixsize=SIMLIB_PIXSIZE,
    simlib_npe_pixel_saturate=SIMLIB_NPE_PIXEL_SATURATE,
    simlib_photflag_saturate=SIMLIB_PHOTFLAG_SATURATE,
    simlib_psf_unit=SIMLIB_PSF_UNIT,
    typical_days_by_type=TYPICAL_DAYS_BY_TYPE,
    default_typical_days=DEFAULT_TYPICAL_DAYS,
    ra_col=RA_COL,
    dec_col=DEC_COL,
    disc_col=DISC_COL,
    name_col=NAME_COL,
    type_col=TYPE_COL,
    allow_filter_changes_in_twilight=ALLOW_FILTER_CHANGES_IN_TWILIGHT,
)

cfg

PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, twilight_sun_alt_min_deg=-14.0, twilight_sun_alt_max_deg=-8.0, filters=['g', 'r', 'i', 'z'], carousel_capacity=5, filter_change_s=120.0, readout_s=2.0, inter_exposure_min_s=15.0, filter_change_time_s=None, readout_time_s=None, exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, max_filters_per_visit=3, start_filter='g', sun_alt_policy=[(-18.0, -15.0, ['g', 'r', 'i', 'z', 'y']), (-15.0, -12.0, ['r', 'i', 'z', 'y']), (-12.0, 0.0, ['i', 'r', 'z', 'y'])], sun_alt_exposure_ladder=[], slew_small_deg=3.5, slew_small_time_s=4.0, slew_rate_deg_per_s=5.25, slew_settle_s=1.0, min_moon_sep_by_filter={'u': 80.0, 'g': 50.0, 'r': 35.0, 'i': 30.0, 'z': 25.0, 'y': 20.0}, require_single_time_for_all_filters=True, per_sn_cap_s=120.0, morning_cap_s='auto', evening_cap_s='auto', morning_twilight=None, evening_twilight=None, twilight_step_min=2, max_sn_per_night=99999, hybrid_detections=2, hybrid_exposure_s=300.0, lc

## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`CSV_PATH`, writes output CSVs to `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [3]:
# Execute the planner and return data frames
perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    run_label=RUN_LABEL,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)

RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/366 [00:00<?, ?night/s]

2023-01-01: eligible=725 visible=509 planned_total=174
  evening_twilight: 2023-01-02T00:27:00+00:00 → 2023-01-02T01:00:00+00:00 (local 19:44 → 20:17 UTC-04:43)
  morning_twilight: 2023-01-02T08:34:00+00:00 → 2023-01-02T09:07:00+00:00 (local 03:51 → 04:24 UTC-04:43)
2023-01-02: eligible=734 visible=533 planned_total=171
  evening_twilight: 2023-01-03T00:27:00+00:00 → 2023-01-03T01:00:00+00:00 (local 19:44 → 20:17 UTC-04:43)
  morning_twilight: 2023-01-03T08:35:00+00:00 → 2023-01-03T09:07:00+00:00 (local 03:52 → 04:24 UTC-04:43)
2023-01-03: eligible=734 visible=531 planned_total=163
  evening_twilight: 2023-01-04T00:27:00+00:00 → 2023-01-04T01:00:00+00:00 (local 19:44 → 20:17 UTC-04:43)
  morning_twilight: 2023-01-04T08:35:00+00:00 → 2023-01-04T09:08:00+00:00 (local 03:52 → 04:25 UTC-04:43)
2023-01-04: eligible=727 visible=526 planned_total=162
  evening_twilight: 2023-01-05T00:27:00+00:00 → 2023-01-05T01:00:00+00:00 (local 19:44 → 20:17 UTC-04:43)
  morning_twilight: 2023-01-05T08:36:0

,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,visit_start_utc,sn_end_utc,filter,t_exp_s,...,cadence_gate_passed,slew_s,cross_filter_change_s,filter_changes_s,readout_s,exposure_s,guard_s,inter_exposure_guard_enforced,total_time_s,elapsed_overhead_s
0,2023-01-01,evening,2022adgf,26.513167,-21.206239,2023-01-02T00:27:00+00:00,2023-01-02T00:27:00+00:00,2023-01-02T00:31:07+00:00,r,5.0,...,True,0.00,120.0,120.0,2.0,5.0,0.00,False,247.0,242.00
1,2023-01-01,evening,2022yuh,25.110615,-26.051126,2023-01-02T00:27:00+00:00,2023-01-02T00:31:07+00:00,2023-01-02T00:31:27+00:00,r,5.0,...,True,5.29,0.0,0.0,2.0,5.0,9.71,True,20.0,5.29
2,2023-01-01,evening,22fkuqa,19.684129,-23.528497,2023-01-02T00:27:00+00:00,2023-01-02T00:31:27+00:00,2023-01-02T00:31:47+00:00,r,5.0,...,True,5.39,0.0,0.0,2.0,5.0,9.61,True,20.0,5.39
3,2023-01-01,evening,2022aeay,15.550774,-20.622240,2023-01-02T00:27:00+00:00,2023-01-02T00:31:47+00:00,2023-01-02T00:32:07+00:00,r,5.0,...,True,5.25,0.0,0.0,2.0,5.0,9.75,True,20.0,5.25
4,2023-01-01,evening,2022acjg,13.602458,-15.604381,2023-01-02T00:27:00+00:00,2023-01-02T00:32:07+00:00,2023-01-02T00:32:27+00:00,r,5.0,...,True,5.35,0.0,0.0,2.0,5.0,9.65,True,20.0,5.35
5,2023-01-01,evening,22fksnp,19.735824,-12.962475,2023-01-02T00:27:00+00:00,2023-01-02T00:32:27+00:00,2023-01-02T00:32:47+00:00,r,5.0,...,True,5.57,0.0,0.0,2.0,5.0,9.43,True,20.0,5.57
6,2023-01-01,evening,22fkqid,19.735824,-12.962475,2023-01-02T00:27:00+00:00,2023-01-02T00:32:47+00:00,2023-01-02T00:33:07+00:00,r,5.0,...,True,0.00,0.0,0.0,2.0,5.0,13.00,True,20.0,2.00
7,2023-01-01,evening,22fkzln,19.735824,-12.962475,2023-01-02T00:27:00+00:00,2023-01-02T00:33:07+00:00,2023-01-02T00:33:27+00:00,r,5.0,...,True,0.00,0.0,0.0,2.0,5.0,13.00,True,20.0,2.00
8,2023-01-01,evening,22fkdoq,21.193116,-11.964294,2023-01-02T00:27:00+00:00,2023-01-02T00:33:27+00:00,2023-01-02T00:33:47+00:00,r,5.0,...,True,5.00,0.0,0.0,2.0,5.0,10.00,True,20.0,5.00
9,2023-01-01,evening,2022ackp,25.431834,-14.633520,2023-01-02T00:27:00+00:00,2023-01-02T00:33:47+00:00,2023-01-02T00:34:07+00:00,r,5.0,...,True,5.27,0.0,0.0,2.0,5.0,9.73,True,20.0,5.27


In [4]:
# Show the nightly summary
nights_df

,date,twilight_window,n_candidates,n_planned,unique_targets_observed,repeat_fraction,sum_time_s,window_cap_s,swap_count,internal_filter_changes,...,cap_source,median_sky_mag_arcsec2,median_alt_deg,cad_median_abs_err_by_filter_csv,cad_within_pct_by_filter_csv,cad_median_abs_err_all_d,cad_within_pct_all,quota_assigned,n_candidates_pre_cap,n_candidates_post_cap
0,2023-01-01,evening,285,87,87,0.000,1967.0,1980,1,0,...,window_duration,19.202325,62.790,,,NaN,NaN,50000,285,285
1,2023-01-01,morning,224,87,87,0.000,1967.0,1980,1,0,...,window_duration,19.010150,47.170,,,NaN,NaN,49999,224,224
2,2023-01-02,evening,305,87,87,0.000,1967.0,1980,1,0,...,window_duration,19.195122,39.770,,,NaN,NaN,50769,305,305
3,2023-01-02,morning,228,84,84,0.000,1907.0,1920,1,0,...,window_duration,19.372960,46.730,,,NaN,NaN,49230,228,228
4,2023-01-03,evening,302,76,76,0.000,1978.2,1980,2,0,...,window_duration,19.508347,67.265,,,NaN,NaN,50000,302,302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2023-12-30,morning,157,64,64,0.000,1978.3,1980,3,0,...,window_duration,19.371033,55.685,"i:6.51,r:6.0,z:4.0","i:6.2,r:33.3,z:0.0",4.011,15.0,49999,157,157
728,2023-12-31,evening,308,64,64,0.000,1979.0,1980,3,0,...,window_duration,19.572816,63.830,"i:6.01,r:4.01","i:0.0,r:0.0",6.008,0.0,50000,308,308
729,2023-12-31,morning,166,69,67,0.029,1850.4,1980,2,0,...,window_duration,19.322374,52.820,"i:3.01,r:17.01","i:32.4,r:16.0",5.017,25.8,49999,166,166
730,2024-01-01,evening,319,76,76,0.000,1977.4,1980,2,0,...,window_duration,18.980022,67.260,"r:4.66,z:0.5","r:0.0,z:50.0",0.992,42.9,50000,319,319



## Inspect outputs on disk


In [5]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

['../twilight_outputs/lsst_twilight_plan_HYBRID_strategy_2022-01-01_to_2022-01-05.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01T00:00:00+00:00_to_2024-01-01T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2023-01-01_to_2023-01-20.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2024-01-01T00:00:00+00:00_to_2024-01-03T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_plan_hybrid_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_plan_one_dec_2021-01-01_to_2021-01-20.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_HYBRID_strategy_2022-01-01_to_2022-01-05.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01T00:00:00+00:00_to_2024-01-01T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2023-01-01_to_2023-01-20.csv',
 '../twilight_outputs/lsst_twilight_sequence_true_hybrid_2024-01-01T00:00:00+00:00_to_2024-01-03T00:00:00+00:00.csv',
 '../twilight_outputs/lsst_twilight